In [1]:
# code to show how to calculate a smooth long-term daily mean
# Sample data: OLR
# steps: 
#
#       1) read daily OLR at a particular location to get a time series
#       2) Create long-term climatology for each day of the year 
#       3) Remove all harmonics except 0 (DC), 1 (annual) and 2 (semi-annual)
#       4) Subtract the daily mean from the original OLR time series
#
#

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from datetime import date
import xarray as xr


In [3]:
year_start = 1982
month_start = 1
day_start   = 1
year_end = 2022
month_end = 12
day_end   = 31

In [4]:
data_dir  = '~/work/aiyyer/data/olr/'
olr_file  =  'olr.day.mean.nc'
file_path = data_dir+olr_file
ds = xr.open_dataset(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/glade/u/home/aiyyer/work/aiyyer/data/olr/olr.day.mean.nc'

In [ ]:
olr = ds.olr.sel( lat = 10., lon = 0. , 
                 time = slice(date(year_start,month_start,day_start),date(year_end,month_end,day_end) ) ) 
olr.plot()

In [ ]:
olr.time

In [ ]:
#olr = olr.convert_calendar('365_day')

In [ ]:
daily_mean = olr.groupby(olr.time.dt.dayofyear).mean()

In [ ]:

# Fourier transform
fft_result = fft(daily_mean.values)

# Retain only the first 2 harmonics
fft_result[3:] = 0  # Set higher harmonics to zero (keep only DC, fundamental, and second harmonic)

# Inverse Fourier transform
filtered_daily_mean = ifft(fft_result).real


In [ ]:

# Plot the original and filtered daily mean
plt.figure(figsize=(10, 6))
plt.plot(daily_mean, label='Original Daily Mean')
plt.plot(filtered_daily_mean, label='Filtered Daily Mean (2 harmonics)')
plt.xlabel('Day of Year')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
filtered_daily_mean = daily_mean.copy(data=filtered_daily_mean)

In [ ]:
olr_anom  = olr.groupby(olr.time.dt.dayofyear) - filtered_daily_mean

In [ ]:
olr.sel(time = slice(date(2019,1,1),date(2019,12,31) ) ). plot()
olr_anom.sel(time = slice(date(2019,1,1),date(2019,12,31) ) ). plot()

In [ ]:
diff = olr.sel(time = slice(date(2019,1,1),date(2019,12,31) ) ) - olr_anom.sel(time = slice(date(2019,1,1),date(2019,12,31) ) )
diff.plot()